# Imports and spark config

In [28]:
import os

# Set SPARK_HOME
os.environ["SPARK_HOME"] = r"C:/Spark/spark-3.5.0-bin-hadoop3"

# Set PYTHONPATH
spark_home = os.environ.get("SPARK_HOME", "")
if spark_home:
    python_path = os.path.join(spark_home, "python")
    py4j_zip = os.path.join(spark_home, "python", "lib", "py4j-0.10.9.7-src.zip")
    os.environ["PYTHONPATH"] = f"{python_path};{py4j_zip};{os.environ.get('PYTHONPATH', '')}"

# Set PYSPARK_PYTHON
os.environ["PYSPARK_PYTHON"] = r"C:\Users\Vincenzo\AppData\Local\Programs\Python\Python38\python.exe"
print(os.environ.get('PYSPARK_PYTHON'))

# Set PATH
if spark_home:
    os.environ["PATH"] = f"{python_path};{os.path.join(spark_home, 'bin')};{os.environ.get('PATH', '')}"

C:\Users\Vincenzo\AppData\Local\Programs\Python\Python38\python.exe


In [29]:
import findspark
findspark.init()

import pandas as pd
from tqdm import tqdm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import gc

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [30]:
spark = SparkSession.builder.appName("spark").config("spark.driver.memory", "24g").getOrCreate()
spark

In [31]:
# spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
# spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 5)

In [32]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Data preparation

## Dataset import

In [33]:
df = spark.read.parquet(r"c:\Users\Vincenzo\PROJECTS\DDAM_data\brewery\brewery_data_complete_extended.parquet")
type(df)

pyspark.sql.dataframe.DataFrame

In [34]:
df.printSchema()
rows = df.count()
cols = len(df.columns)

print(f'Dimension of the Dataframe is: {(rows,cols)}')
df.show(5)

root
 |-- Batch_ID: long (nullable = true)
 |-- Brew_Date: string (nullable = true)
 |-- Beer_Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Fermentation_Time: long (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- pH_Level: double (nullable = true)
 |-- Gravity: double (nullable = true)
 |-- Alcohol_Content: double (nullable = true)
 |-- Bitterness: long (nullable = true)
 |-- Color: long (nullable = true)
 |-- Ingredient_Ratio: string (nullable = true)
 |-- Volume_Produced: long (nullable = true)
 |-- Total_Sales: double (nullable = true)
 |-- Quality_Score: double (nullable = true)
 |-- Brewhouse_Efficiency: double (nullable = true)
 |-- Loss_During_Brewing: double (nullable = true)
 |-- Loss_During_Fermentation: double (nullable = true)
 |-- Loss_During_Bottling_Kegging: double (nullable = true)

Dimension of the Dataframe is: (10000000, 20)
+--------+-------------------+----------+----+------------+--

## Sampling and df assignments

### Sampling

In [35]:

summary_stats = df.sample(False, 0.1, seed=10).toPandas().describe()
rounded_describe = np.round(summary_stats, 2)
rounded_describe

,Batch_ID,Fermentation_Time,Temperature,pH_Level,Gravity,Alcohol_Content,Bitterness,Color,Volume_Produced,Total_Sales,Quality_Score,Brewhouse_Efficiency,Loss_During_Brewing,Loss_During_Fermentation,Loss_During_Bottling_Kegging
count,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00,1000930.00
mean,4999453.59,14.50,20.00,5.00,1.05,5.25,39.51,12.00,2746.26,10504.09,8.00,79.99,3.00,3.00,3.00
std,2889162.47,2.87,2.89,0.29,0.01,0.43,11.55,4.32,1298.34,5486.58,1.15,5.78,1.16,1.15,1.15
min,1.00,10.00,15.00,4.50,1.03,4.50,20.00,5.00,500.00,1000.03,6.00,70.00,1.00,1.00,1.00
25%,2496035.25,12.00,17.50,4.75,1.04,4.87,29.00,8.00,1622.00,5757.46,7.00,74.99,2.00,2.00,2.00
50%,4999849.00,14.00,20.00,5.00,1.05,5.25,40.00,12.00,2743.00,10504.97,8.00,79.99,3.00,3.00,3.00
75%,7504892.00,17.00,22.50,5.25,1.07,5.62,50.00,16.00,3869.00,15259.37,9.00,85.00,4.00,4.00,4.00
max,9999997.00,19.00,25.00,5.50,1.08,6.00,59.00,19.00,4999.00,20000.00,10.00,90.00,5.00,5.00,5.00


#### Attempted memory managment

In [36]:
# import gc

# sample.unpersist()
# del summary_stats
# gc.collect()

### New columns

In [37]:
from pyspark.sql.functions import split, col

# Split the ratio_column into three parts based on the colon delimiter
split_col = split(col("Ingredient_Ratio"), ":")

# Create three separate columns for water, grains, and hops
df = df.withColumn("water", split_col.getItem(0).cast("double"))
df = df.withColumn("grains", split_col.getItem(1).cast("double"))
df = df.withColumn("hops", split_col.getItem(2).cast("double"))
df = df.drop("Ingredient_Ratio")

In [38]:
# Define a new column with the USD per liter ratio and brewing efficiency
df = df.withColumn("USD_per_Liter", F.col("Total_Sales") / F.col("Volume_Produced").cast("double"))
df = df.withColumn("Brewing_efficency", F.col("Brewhouse_Efficiency") / F.col("Volume_Produced").cast("double"))

In [39]:
from pyspark.sql.types import StringType

# Identify numerical and non-numerical columns
numerical_cols = [col_name for col_name, data_type in df.dtypes if data_type != 'string']
categorical_cols = [col_name for col_name, data_type in df.dtypes if data_type == 'string']

print(f'Numerical columns: {numerical_cols}')
print(f'Categorical columns: {categorical_cols}')

Numerical columns: ['Batch_ID', 'Fermentation_Time', 'Temperature', 'pH_Level', 'Gravity', 'Alcohol_Content', 'Bitterness', 'Color', 'Volume_Produced', 'Total_Sales', 'Quality_Score', 'Brewhouse_Efficiency', 'Loss_During_Brewing', 'Loss_During_Fermentation', 'Loss_During_Bottling_Kegging', 'water', 'grains', 'hops', 'USD_per_Liter', 'Brewing_efficency']
Categorical columns: ['Brew_Date', 'Beer_Style', 'SKU', 'Location']


In [40]:
# distinct_num_counts = {}

# for col in numerical_cols:
#     distinct_count = df.select(col).distinct().count()
#     distinct_num_counts[col] = distinct_count

#     print(f"- {col} - {distinct_count} distinct values")

In [41]:
# distinct_cat_counts = {}

# for col in categorical_cols:
#     distinct_count = df.select(col).distinct().count()
#     distinct_cat_counts[col] = distinct_count

#     print(f"- {col} - {distinct_count} distinct values")

# Outlier detection

### Prep

In [42]:
df.columns

['Batch_ID',
 'Brew_Date',
 'Beer_Style',
 'SKU',
 'Location',
 'Fermentation_Time',
 'Temperature',
 'pH_Level',
 'Gravity',
 'Alcohol_Content',
 'Bitterness',
 'Color',
 'Volume_Produced',
 'Total_Sales',
 'Quality_Score',
 'Brewhouse_Efficiency',
 'Loss_During_Brewing',
 'Loss_During_Fermentation',
 'Loss_During_Bottling_Kegging',
 'water',
 'grains',
 'hops',
 'USD_per_Liter',
 'Brewing_efficency']

In [43]:
from pyspark.sql.functions import lower
columns = df.columns
filtered_columns = [col for col in columns if df.select(lower(col)).distinct().count() > 10]

filtered_columns.remove('Brew_Date')
filtered_columns.remove('Batch_ID')
print(filtered_columns)

['Temperature', 'pH_Level', 'Gravity', 'Alcohol_Content', 'Bitterness', 'Color', 'Volume_Produced', 'Total_Sales', 'Quality_Score', 'Brewhouse_Efficiency', 'Loss_During_Brewing', 'Loss_During_Fermentation', 'Loss_During_Bottling_Kegging', 'grains', 'hops', 'USD_per_Liter', 'Brewing_efficency']


### Skewennes - Kurtosis

In [44]:
from pyspark.sql.functions import skewness, kurtosis

for col in filtered_columns:
    df.select(skewness(col), kurtosis(col)).show(truncate=False) 

+---------------------+---------------------+
|skewness(Temperature)|kurtosis(Temperature)|
+---------------------+---------------------+
|2.4518519193579075E-4|-1.2001740629835786  |
+---------------------+---------------------+

+--------------------+------------------+
|skewness(pH_Level)  |kurtosis(pH_Level)|
+--------------------+------------------+
|5.810463584429394E-4|-1.199723583471544|
+--------------------+------------------+

+---------------------+------------------+
|skewness(Gravity)    |kurtosis(Gravity) |
+---------------------+------------------+
|-3.541412548478481E-5|-1.200077666104571|
+---------------------+------------------+

+-------------------------+-------------------------+
|skewness(Alcohol_Content)|kurtosis(Alcohol_Content)|
+-------------------------+-------------------------+
|6.555985166403937E-4     |-1.199645816573591       |
+-------------------------+-------------------------+

+--------------------+--------------------+
|skewness(Bitterness)|kurto

In [45]:
high_kurt_col = ["USD_per_Liter","Brewing_efficency"]

Absolutely! Let's analyze the skewness and kurtosis values you've provided:

Understanding Skewness

    Positive Skewness: Indicates a tail stretching towards the right of the distribution (more observations with higher values).
    Negative Skewness: Indicates a tail stretching towards the left of the distribution (more observations with lower values).
    Zero (or Near-Zero) Skewness: Suggests a relatively symmetrical distribution.

Understanding Kurtosis

    Normal Distribution (Mesokurtic): Has a kurtosis around 0.
    High Kurtosis (Leptokurtic): Indicates a sharper peak and heavier tails compared to a normal distribution (more outliers).
    Low Kurtosis (Platykurtic): Indicates a flatter peak and lighter tails compared to a normal distribution (fewer outliers).

Analyzing Your Data

Here's a breakdown of what the skewness and kurtosis values suggest about your features:

Generally:

    Skewness: Most of your features have slightly positive skewness, meaning the data distribution might lean slightly towards higher values. However, the skewness values are close to zero for many features, suggesting relatively balanced distributions.
    Kurtosis: All your features exhibit negative kurtosis, indicating that they might have flatter distributions than a standard normal distribution with fewer outliers.

#### Skeweness visualization - when needed

In [46]:
# import statsmodels.api as sm
# import matplotlib.pyplot as plt
# import seaborn as sns

# pdf = df.select('color').toPandas()

# # Create the count plot using seaborn
# import seaborn as sns
# sns.countplot(x='color', data=pdf, order=pdf['color'].value_counts().index) 
# plt.title("Distribution of Color Feature")
# plt.xticks(rotation=45)  # Rotate x labels if they are long
# plt.show()

# # Density Plot
# sns.kdeplot(pdf['color'])
# plt.title("Density Plot of Color Feature (Negative Skew)")
# plt.show()


### Quantile-Quantile plot

In [47]:
# #more sparky approach

# import pyspark.sql.functions as F
# from pyspark.sql.types import DoubleType
# import matplotlib.pyplot as plt

# def q_q_plot_pyspark(df, filtered_columns):
#     for col in filtered_columns:
#         quantiles = df.approxQuantile(col, [0.0, 0.05, 0.25, 0.5, 0.75, 0.95, 1.0], 0.0)  

#         # Sample data for plotting on the theoretical normal distribution
#         theoretical_quantiles = F.array([F.lit(i) for i in quantiles]).cast(DoubleType())
#         std_norm_quantiles = F.expr("percentile_approx(array(0.0, 0.05, 0.25, 0.5, 0.75, 0.95, 1.0), 0.5)")

#         # QQ plot comparison
#         qqplot_data = df.select(col, theoretical_quantiles, std_norm_quantiles)

#         # Convert Spark DataFrame to Pandas, this will likely be more manageable
#         pandas_df = qqplot_data.toPandas()

#         # Plotting
#         plt.figure()
#         plt.scatter(pandas_df[col], pandas_df['std_norm_quantiles'])
#         plt.xlabel(f"Sample Quantiles ({col})")
#         plt.ylabel("Theoretical Normal Quantiles")
#         plt.title(f"QQ Plot for Column: {col}")
#         plt.show()

# q_q_plot_pyspark(df, filtered_columns) 

In [48]:

# import statsmodels.api as sm
# import matplotlib.pyplot as plt

# def q_q_plot(df, filtered_columns):
#     for col in filtered_columns:
#         fig, ax = plt.subplots()
#         sm.qqplot(df.select(col).toPandas()[col], line='s', ax=ax)

#         # Add title with column name
#         ax.set_title(f"QQ Plot for Column: {col}") 

#         plt.show()

# q_q_plot(df, filtered_columns) 

Interpreting the Q-Q Plot:

    Normal Distribution: If the data points closely follow the diagonal line, the distribution is likely close to normal.
    Negative Skew (Long Left Tail): If the points curve below the line on the left side, it indicates a negative skew.
    Positive Skew (Long Right Tail): If the points curve above the line on the right side, it indicates a positive skew.

### Transformations

- not needed misinterpreted results from skewenss due to truncated show df

The F.log1p() function is a PySpark function that calculates the natural logarithm of a column value plus one. It is commonly used to transform skewed or heavily right-skewed data to a more normalized distribution. By adding one to each value before taking the logarithm, it ensures that the function can handle zero or negative values without throwing an error.

In [49]:
# high_skew_col = ["pH_Level", "Gravity", "Alcohol_Content"]

# transformed_df = df.alias("transformed_df")

# transformed_df = transformed_df.withColumn("pH_Level", F.log1p("pH_Level"))
# transformed_df = transformed_df.withColumn("Gravity", F.log1p("Gravity"))
# transformed_df = transformed_df.withColumn("Alcohol_Content", F.log1p("Alcohol_Content"))

# df.select(high_skew_col).show(5)
# transformed_df.select(high_skew_col).show(5)

### Z-score

#### z-score

['Temperature', 'pH_Level', 'Gravity', 'Alcohol_Content', 'Bitterness', 'Color', 'Ingredient_Ratio', 'Volume_Produced', 'Total_Sales', 'Quality_Score', 'Brewhouse_Efficiency', 'Loss_During_Brewing', 'Loss_During_Fermentation', 'Loss_During_Bottling_Kegging']

##### threshold 3.0
- Number of Outliers based on Temperature: 0
- Number of Outliers based on pH_Level: 0
- Number of Outliers based on Gravity: 0
- Number of Outliers based on Alcohol_Content: 0
- Number of Outliers based on Bitterness: 0
- Number of Outliers based on Color: 0
- Number of Outliers based on Volume_Produced: 0
- Number of Outliers based on Total_Sales: 0
- Number of Outliers based on Quality_Score: 0
- Number of Outliers based on Brewhouse_Efficiency: 0
- Number of Outliers based on Loss_During_Brewing: 0
- Number of Outliers based on Loss_During_Fermentation: 0
- Number of Outliers based on Loss_During_Bottling_Kegging: 0
- Number of Outliers based on grains: 0
- Number of Outliers based on hops: 0
- Number of Outliers based on USD_per_Liter: 250211
- Number of Outliers based on Brewing_efficency: 247848

##### Threshold 2.0 / 2.5 - (same)

In [50]:
from pyspark.sql.functions import col, abs, sqrt, avg, pow, when

def detect_outliers_zscore(df, feature_col, threshold=3.0):
    mean_val = df.select(avg(df[feature_col])).collect()[0][0] 
    std_dev = df.select(sqrt(avg(pow(df[feature_col] - mean_val, 2)))).collect()[0][0]  

    return df.withColumn("outlier_" + feature_col, 
                          when(abs((df[feature_col] - mean_val) / std_dev) > threshold, 1).otherwise(0))  

In [51]:
for col in filtered_columns:
    outlier = detect_outliers_zscore(df, col)
    outlier_col_name = f"outlier_{col}"

    # Filtering directly on the 'outlier' DataFrame 
    outlier_count = outlier.filter(outlier[outlier_col_name] == 1).count()
    print(f"Number of Outliers based on {col}: {outlier_count}")

Number of Outliers based on Temperature: 0
Number of Outliers based on pH_Level: 0
Number of Outliers based on Gravity: 0
Number of Outliers based on Alcohol_Content: 0
Number of Outliers based on Bitterness: 0
Number of Outliers based on Color: 0
Number of Outliers based on Volume_Produced: 0
Number of Outliers based on Total_Sales: 0
Number of Outliers based on Quality_Score: 0
Number of Outliers based on Brewhouse_Efficiency: 0
Number of Outliers based on Loss_During_Brewing: 0
Number of Outliers based on Loss_During_Fermentation: 0
Number of Outliers based on Loss_During_Bottling_Kegging: 0
Number of Outliers based on grains: 0
Number of Outliers based on hops: 0
Number of Outliers based on USD_per_Liter: 250211
Number of Outliers based on Brewing_efficency: 247848


In [52]:
# outlier = detect_outliers_zscore(df, "Bitterness")
# outlier_temp = outlier.filter(col("outlier_Bitterness") == 1).count()
# print(f"Number of Outliers based on Bitterness: {outlier_temp}")

####  modified z-score

['Temperature', 'pH_Level', 'Gravity', 'Alcohol_Content', 'Bitterness', 'Color', 'Ingredient_Ratio', 'Volume_Produced', 'Total_Sales', 'Quality_Score', 'Brewhouse_Efficiency', 'Loss_During_Brewing', 'Loss_During_Fermentation', 'Loss_During_Bottling_Kegging']

##### threshold 3.5
- Number of Outliers based on Temperature: 0
- Number of Outliers based on pH_Level: 0
- Number of Outliers based on Gravity: 0
- Number of Outliers based on Alcohol_Content: 0
- Number of Outliers based on Bitterness: 0
- Number of Outliers based on Color: 0
- Number of Outliers based on Volume_Produced: 0
- Number of Outliers based on Total_Sales: 0
- Number of Outliers based on Quality_Score: 0
- Number of Outliers based on Brewhouse_Efficiency: 0
- Number of Outliers based on Loss_During_Brewing: 0
- Number of Outliers based on Loss_During_Fermentation: 0
- Number of Outliers based on Loss_During_Bottling_Kegging: 0
- Number of Outliers based on grains: 0
- Number of Outliers based on hops: 0
- Number of Outliers based on USD_per_Liter: 428330
- Number of Outliers based on Brewing_efficency: 1397253

In [53]:
# from pyspark.sql.functions import col, abs, median, sqrt, avg, pow

# def detect_outliers_modified_zscore(df, feature_col, threshold=3.5):
#     median_val = df.select(df[feature_col]).collect()[0][0] 
#     mad = df.select(median(abs(df[feature_col] - median_val))).collect()[0][0]

#     return df.withColumn("outlier_" + feature_col, 
#                           when(abs(0.6745 * (df[feature_col] - median_val) / mad) > threshold, 1).otherwise(0))

In [54]:
# for col in filtered_columns:
#     outlier = detect_outliers_modified_zscore(df, col)
#     outlier_col_name = f"outlier_{col}"

#     # Filtering directly on the 'outlier' DataFrame 
#     outlier_count = outlier.filter(outlier[outlier_col_name] == 1).count()
#     print(f"Number of Outliers based on {col}: {outlier_count}")

####  modified z-score - adaptive threshold - i doubt the statistical right of it

['Temperature', 'pH_Level', 'Gravity', 'Alcohol_Content', 'Bitterness', 'Color', 'Volume_Produced', 'Total_Sales', 'Quality_Score', 'Brewhouse_Efficiency', 'Loss_During_Brewing', 'Loss_During_Fermentation', 'Loss_During_Bottling_Kegging']

##### threshold 3.5
- Number of Outliers based on Temperature: 0
- Number of Outliers based on pH_Level: 3556568
- Number of Outliers based on Gravity: 9535830
- Number of Outliers based on Alcohol_Content: 189356
- Number of Outliers based on Bitterness: 0
- Number of Outliers based on Color: 0
- Number of Outliers based on Volume_Produced: 0
- Number of Outliers based on Total_Sales: 0
- Number of Outliers based on Quality_Score: 0
- Number of Outliers based on Brewhouse_Efficiency: 0
- Number of Outliers based on Loss_During_Brewing: 0
- Number of Outliers based on Loss_During_Fermentation: 0
- Number of Outliers based on Loss_During_Bottling_Kegging: 0


In [55]:
def detect_outliers_modified_zscore_adaptive(df, feature_col, threshold_factor=2.0):

    median_val = df.select(df[feature_col]).collect()[0][0] 
    mad = df.select(median(abs(df[feature_col] - median_val))).collect()[0][0]

    std_dev = np.std(df.select(feature_col).toPandas()[feature_col])  # Assuming numerical column 
    adaptive_threshold =  threshold_factor * std_dev

    return df.withColumn("outlier_" + feature_col, 
                          when(abs(0.6745 * (df[feature_col] - median_val) / mad) > adaptive_threshold, 1).otherwise(0))

In [57]:
# filtered_columns.remove("Ingredient_Ratio")

# for col in filtered_columns:
#     outlier = detect_outliers_modified_zscore_adaptive(df, col)
#     outlier_col_name = f"outlier_{col}"

#     # Filtering directly on the 'outlier' DataFrame 
#     outlier_count = outlier.filter(outlier[outlier_col_name] == 1).count()
#     print(f"Number of Outliers based on {col}: {outlier_count}")

# Feature analysis

### Low-values continous variables analysis

#### Functions

In [59]:
# functions

from pyspark.sql.functions import when, col
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, dense_rank, when
from pyspark.sql.window import Window

def ordinal_encode_pyspark(df, input_col, output_col):
    """ Ordinal encodes a column within a PySpark DataFrame

    Args:
        df (pyspark.sql.DataFrame): The input Spark DataFrame.
        input_col (str): Name of the column to be encoded. Defaults to "Bitterness".
        output_col (str): Name for the new ordinal encoded column. Defaults to "Bitterness_ordinal".
    
    Returns:
        pyspark.sql.DataFrame: The Spark DataFrame with the ordinal encoded column.
    """

    window = Window.orderBy(input_col) 
    return df.withColumn(output_col, dense_rank().over(window)) \
             .withColumn(output_col, when(col(input_col).isNull(), 0).otherwise(col(output_col))) \
             #.drop(input_col)
             
def check_distinct_values(df, feature_col):
    """  
    Prints the distinct values of a feature in a PySpark DataFrame.

    Args:
        df (pyspark.sql.DataFrame): The PySpark DataFrame.
        feature_col (str): The name of the feature column.
    """

    distinct_values = df.select(feature_col).distinct().collect()

    print(f"Distinct values in '{feature_col}':")
    for row in distinct_values:
        print(row[feature_col]) 

    # functions

def one_hot_encode_column(df, input_col, output_col_prefix="encoded"):
    """ One-hot encodes a specified categorical column, drops intermediary columns,
        and allows for specifying the prefix of the output columns.

    Args:
        df (pyspark.sql.DataFrame): Input DataFrame.
        input_col (str): The name of the column containing categorical data.
        output_col_prefix (str): Prefix for the generated one-hot encoded column names.

    Returns:
        pyspark.sql.DataFrame: DataFrame with the one-hot encoded column(s) and 
                               intermediary columns removed.
    """

    indexer = StringIndexer(inputCol=input_col, outputCol=input_col + "_index")
    encoder = OneHotEncoder(inputCol=input_col + "_index", outputCol=output_col_prefix + input_col + "features")
    assembler = VectorAssembler(inputCols=[input_col + "_index"], outputCol="features") 

    pipeline = Pipeline(stages=[indexer, assembler, encoder])
    transformed_df = pipeline.fit(df).transform(df)

    # Drop unnecessary columns
    drop_cols = [input_col + "_index", "features"]
    return transformed_df.drop(*drop_cols)  # Dynamic column dropping

def discretize_and_encode(df, input_col="fermentation_time"):
    """ Discretizes a feature into bins, one-hot encodes the bins, and preserves the original column

    Args:
        df (pyspark.sql.DataFrame): Input DataFrame
        input_col (str): Name of the column to be discretized and encoded

    Returns:
        pyspark.sql.DataFrame:  DataFrame with new discretization and encoded columns
    """

    df = df.withColumn(input_col + "_bin", 
                       when((col(input_col) >= 10) & (col(input_col) <= 13), "Short")
                       .when((col(input_col) >= 14) & (col(input_col) <= 16), "Medium")
                       .otherwise("Long"))
    
    return one_hot_encode_column(df, input_col=input_col + "_bin") 

#### Color

In [ ]:
color_sample = df.select("Color").sample(fraction=0.1, seed=42).alias("color_sample")
color_frequency = color_sample.groupBy("Color").count().toPandas()

plt.figure(figsize=(10, 5))
sns.barplot(data=color_frequency, x="Color", y="count", order=color_frequency.sort_values('count', ascending=False)["Color"])
plt.title('Frequency of Color in the Dataset')
plt.xlabel('Color')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# sample = ordinal_encode_pyspark(sample, "Color", "Color_ordinal")
# sample.show(5)

# summary_stats = sample.select("Color_ordinal").toPandas().describe()
# rounded_describe = np.round(summary_stats, 2)
# rounded_describe

In [ ]:
# color_frequency = sample.groupBy("Color_ordinal").count().toPandas()

# plt.figure(figsize=(10, 5))
# sns.barplot(data=color_frequency, x="Color_ordinal", y="count", order=color_frequency.sort_values('Color_ordinal', ascending=True)["Color_ordinal"])
# plt.title('Frequency of Color in the Dataset')
# plt.xlabel('Color')
# plt.ylabel('Frequency')
# plt.xticks(rotation=90)
# plt.show()


#### Bitterness

In [ ]:
bitterness_sample = df.select("Bitterness").sample(fraction=0.1, seed=42).alias("bitterness_sample")
Bitterness_frequency = bitterness_sample.groupBy("Bitterness").count().toPandas()

plt.figure(figsize=(10, 5))
sns.barplot(data=Bitterness_frequency, x="Bitterness", y="count", order=Bitterness_frequency.sort_values('Bitterness', ascending=True)["Bitterness"])
plt.title('Frequency of Bitterness in the Dataset')
plt.xlabel('Bitterness')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()


In [ ]:
# sample = ordinal_encode_pyspark(sample, "Bitterness", "Bitterness_ordinal") 
# sample.show(5)

# summary_stats = sample.select("Bitterness_ordinal").toPandas().describe()
# rounded_describe = np.round(summary_stats, 2)
# rounded_describe

In [ ]:
# Bitterness_ordinal_frequency = sample.groupBy("Bitterness_ordinal").count().toPandas()

# plt.figure(figsize=(10, 5))
# sns.barplot(data=Bitterness_ordinal_frequency, x="Bitterness_ordinal", y="count", order=Bitterness_ordinal_frequency.sort_values('Bitterness_ordinal', ascending=True)["Bitterness_ordinal"])
# plt.title('Frequency of Bitterness_ordinal in the Dataset')
# plt.xlabel('Bitterness_ordinal')
# plt.ylabel('Frequency')
# plt.xticks(rotation=90)
# plt.show()


#### Grains

In [ ]:
grain_sample = df.select("grains").sample(fraction=0.1, seed=42).alias("grain_sample")
grains_frequency = grain_sample.groupBy("grains").count().toPandas()

plt.figure(figsize=(10, 5))
sns.barplot(data=grains_frequency, x="grains", y="count", order=grains_frequency.sort_values('grains', ascending=True)["grains"])
plt.title('Frequency of Styles')
plt.xlabel('Beer_Style')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

#### Hops

In [ ]:
hops_sample = df.select("hops").sample(fraction=0.1, seed=42).alias("hops_sample")
hops_frequency = hops_sample.groupBy("hops").count().toPandas()

plt.figure(figsize=(10, 5))
sns.barplot(data=hops_frequency, x="hops", y="count", order=hops_frequency.sort_values('hops', ascending=True)["hops"])
plt.title('Frequency of Hops')
plt.xlabel('Hops')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

#### Fermentation time

In [ ]:
#discretize and encode is hardcoded for fermentation time (!!)

df = discretize_and_encode(df)
df.select("encodedfermentation_time_binfeatures").show(5)

In [ ]:
# df.select("encodedfermentation_time_binfeatures").sample(fraction=0.1, seed= 42).toPandas().describe()

### Categorical variables analysis

#### Beer style

In [60]:
df = one_hot_encode_column(df, "Beer_Style")
df.select("encodedBeer_Stylefeatures").show(5)

+-------------------------+
|encodedBeer_Stylefeatures|
+-------------------------+
|                (7,[],[])|
|            (7,[2],[1.0])|
|                (7,[],[])|
|            (7,[0],[1.0])|
|            (7,[3],[1.0])|
+-------------------------+
only showing top 5 rows



In [61]:
# df.select("encodedBeer_Stylefeatures").sample(fraction=0.1, seed= 42).toPandas().describe()

#### SKU

In [62]:
df = one_hot_encode_column(df, "SKU")
df.select("encodedSKUfeatures").show(5)

+------------------+
|encodedSKUfeatures|
+------------------+
|         (3,[],[])|
|         (3,[],[])|
|         (3,[],[])|
|         (3,[],[])|
|     (3,[2],[1.0])|
+------------------+
only showing top 5 rows



In [63]:
# df.select("encodedSKUfeatures").sample(fraction=0.1, seed= 42).toPandas().describe()

#### Location

In [64]:
df = one_hot_encode_column(df, "Location")
df.select("encodedLocationfeatures").show(5)

+-----------------------+
|encodedLocationfeatures|
+-----------------------+
|              (9,[],[])|
|              (9,[],[])|
|          (9,[7],[1.0])|
|          (9,[4],[1.0])|
|          (9,[3],[1.0])|
+-----------------------+
only showing top 5 rows



In [65]:
# df.select("encodedLocationfeatures").sample(fraction=0.1, seed= 42).toPandas().describe()

In [66]:
gc.collect()

444

# Data Exploration

## Intro statements

In [ ]:
from pyspark.sql.functions import year, month , to_date,  col

plot_sample = df.alias("plot_sample")

plot_sample = plot_sample.withColumn("Year", year("Brew_Date")).withColumn("Month", month("Brew_Date"))
plot_sample = plot_sample.withColumn('Brew_Date', to_date(col('Brew_Date'), 'yyyy-MM-dd HH:mm:ss'))

In [ ]:
import pyspark.sql.functions as F
print(f'We have data in a range from {plot_sample.select(F.min("year")).collect()[0][0]} to {plot_sample.select(F.max("year")).collect()[0][0]}')

## plots

In [ ]:
plot_sample.show(5)

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=plot_sample.select("USD_per_Liter").toPandas(), x="USD_per_Liter", kde=True)

plt.title(f'Histogram for USD_per_Liter')
plt.xlabel("USD_per_Liter")
plt.ylabel('Frequency')
plt.show()

In [ ]:
plot_sample.show(5)

In [ ]:
import matplotlib.pyplot as plt

key_mode_count = plot_sample.groupBy('Color', 'Beer_Style').count().toPandas()

plt.figure(figsize=(10, 5))  # Set the figure size to 10 inches wide and 5 inches tall
sns.barplot(x="Color", y="count", data=key_mode_count, hue='Beer_Style')
plt.show()  # Display the plot

#that is conceptually wrong there cant be the same distribution of type of beers for each color basically impossible

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, to_date, mean
from pyspark.sql.types import DateType
import matplotlib.pyplot as plt
import pandas as pd

sales_by_location_date = plot_sample.groupBy('SKU', 'Brew_Date') \
                           .agg(mean('Total_Sales').alias('Total_Sales')) \
                           .orderBy('Brew_Date')

# Convert Spark DataFrame to pandas for plotting
pandas_df = sales_by_location_date.toPandas()

# Configure plot size
plt.figure(figsize=(20, 6))

# Iterate over locations to create separate trend lines
for SKU in pandas_df['SKU'].unique():
   data = pandas_df[pandas_df['SKU'] == SKU]
   plt.plot(data['Brew_Date'], data['Total_Sales'], label=SKU)

# Customize plot elements (same as before)
plt.xlabel("Brew Date")
plt.ylabel("Total Sales")
plt.title("Sales Trends by SKU")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, to_date, mean
from pyspark.sql.types import DateType
import matplotlib.pyplot as plt
import pandas as pd

sales_by_location_date = plot_sample.groupBy('Beer_Style', 'Brew_Date') \
                           .agg(mean('Total_Sales').alias('Total_Sales')) \
                           .orderBy('Brew_Date')

# Convert Spark DataFrame to pandas for plotting
pandas_df = sales_by_location_date.toPandas()

# Configure plot size
plt.figure(figsize=(20, 6))

# Iterate over locations to create separate trend lines
for Beer_Style in pandas_df['Beer_Style'].unique():
   data = pandas_df[pandas_df['Beer_Style'] == Beer_Style]
   plt.plot(data['Brew_Date'], data['Total_Sales'], label=Beer_Style)

# Customize plot elements (same as before)
plt.xlabel("Brew Date")
plt.ylabel("Total Sales")
plt.title("Sales Trends by Beer_Style")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, to_date,mean
from pyspark.sql.types import DateType
import matplotlib.pyplot as plt
import pandas as pd

sales_by_location_date = plot_sample.groupBy('Location', 'Brew_Date') \
                           .agg(mean('Total_Sales').alias('Total_Sales')) \
                           .orderBy('Brew_Date')

# Convert Spark DataFrame to pandas for plotting
pandas_df = sales_by_location_date.toPandas()

# Configure plot size
plt.figure(figsize=(20, 6))

# Iterate over locations to create separate trend lines
for location in pandas_df['Location'].unique():
   data = pandas_df[pandas_df['Location'] == location]
   plt.plot(data['Brew_Date'], data['Total_Sales'], label=location)

# Customize plot elements (same as before)
plt.xlabel("Brew Date")
plt.ylabel("Total Sales")
plt.title("Sales Trends by Location")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, to_date, mean
from pyspark.sql.types import DateType
import matplotlib.pyplot as plt
import pandas as pd

# Calculation - Aggregate total sales by date 
overall_sales_trend = plot_sample.groupBy('Brew_Date') \
                        .agg(mean('Total_Sales').alias('Total_Sales')) \
                        .orderBy('Brew_Date')

# Convert to pandas and plot
pandas_df = overall_sales_trend.toPandas()

plt.figure(figsize=(20, 6))
plt.plot(pandas_df['Brew_Date'], pandas_df['Total_Sales'])

# Customize plot elements 
plt.xlabel("Brew Date")
plt.ylabel("Total Sales")
plt.title("Overall Sales Trend")
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
plot_sample.show(5)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, to_date, mean
from pyspark.sql.types import DateType
import matplotlib.pyplot as plt
import pandas as pd

plot_sample = plot_sample.withColumn('Brew_Date', to_date(col('Brew_Date'), 'yyyy-MM-dd HH:mm:ss'))

# Calculation - Aggregate total sales by date 
overall_sales_trend = plot_sample.groupBy('Brew_Date') \
                        .agg(mean('Temperature').alias('Temperature')) \
                        .orderBy('Brew_Date')

# Convert to pandas and plot
pandas_df = overall_sales_trend.toPandas()

plt.figure(figsize=(20, 6))
plt.plot(pandas_df['Brew_Date'], pandas_df['Temperature'])

# Customize plot elements 
plt.xlabel("Brew Date")
plt.ylabel("Temperature")
plt.title("Overall Temperature Trend")
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, to_date,mean
from pyspark.sql.types import DateType
import matplotlib.pyplot as plt
import pandas as pd

plot_sample = plot_sample.withColumn('Brew_Date', to_date(col('Brew_Date'), 'yyyy-MM-dd HH:mm:ss'))

sales_by_location_date = plot_sample.groupBy('SKU', 'Brew_Date') \
                           .agg(mean('Fermentation_Time').alias('Fermentation_Time')) \
                           .orderBy('Brew_Date')

# Convert Spark DataFrame to pandas for plotting
pandas_df = sales_by_location_date.toPandas()

# Configure plot size
plt.figure(figsize=(20, 6))

# Iterate over locations to create separate trend lines
for SKU in pandas_df['SKU'].unique():
   data = pandas_df[pandas_df['SKU'] == SKU]
   plt.plot(data['Brew_Date'], data['Fermentation_Time'], label=SKU)

# Customize plot elements (same as before)
plt.xlabel("Brew Date")
plt.ylabel("Fermentation_Time ")
plt.title("Fermentation_Time Trends by SKU")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


### Scatter plots

In [ ]:
plot_sample = plot_sample.sample(fraction=0.1, seed=42)

#### losses against other

- brewhouse efficency (no patterns \ uniform)
- usd per liter (no patterns \ uniform)
- year (no patterns \ uniform)
- month (no patterns \ uniform)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))  # Set the figure size to 10 inches wide and 6 inches tall
sns.scatterplot(data=plot_sample.toPandas(), x="Loss_During_Brewing", y="Brewhouse_Efficiency")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))  # Set the figure size to 10 inches wide and 6 inches tall
sns.scatterplot(data=plot_sample.toPandas(), x="Loss_During_Fermentation", y="Year")
plt.yticks([2020,2021,2022,2023])
plt.xticks(rotation=45) 
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))  # Set the figure size to 12 inches wide and 6 inches tall
sns.scatterplot(data=plot_sample.toPandas(), x="Loss_During_Bottling_Kegging", y="Month")
plt.yticks([1,2,3,4,5,6,7,8,9,10,11,12])
plt.show()


#### quality scores against other

- Color (no patterns \ uniform)
- Alcohol_Content (no patterns \ uniform)
- Bitterness (no patterns \ uniform)
- volume produced (no patterns \ uniform)
- total sales (no patterns \ uniform)
- brewhouse efficency (no patterns \ uniform)
- usd per liter (no patterns \ uniform)


add to the quality score plotting all the features that are strictly related to the fermentation process also aginst losses

In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="USD_per_Liter", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Fermentation_Time", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Temperature", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="pH_Level", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Gravity", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Alcohol_Content", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Bitterness", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Volume_Produced", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Loss_During_Brewing", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Loss_During_Fermentation", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="Loss_During_Bottling_Kegging", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="grains", y="Quality_Score")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6)) 
sns.scatterplot(data=plot_sample.toPandas(), x="hops", y="Quality_Score")
plt.show()


## correlation

In [ ]:
corr_sample = df.sample(fraction=0.1, seed=42)

In [ ]:
corr_sample = corr_sample.drop("water", "Batch_ID","encodedBeer_Stylefeatures", "encodedSKUfeatures", "encodedLocationfeatures")
corr_sample = corr_sample.drop("Brew_Date", "encodedfermentation_time_binfeatures")
corr_sample = corr_sample.drop("Beer_Style", "SKU", "Location", "fermentation_time_bin")
corr_sample.printSchema()

In [ ]:
corr = corr_sample.toPandas()

In [ ]:
corr.corr(method='pearson')
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=2)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
corr.corr(method='spearman')
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=2)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
corr = corr_sample.toPandas().corr(method='kendall')
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=2)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# from pyspark.ml.feature import VectorAssembler

# features = corr_sample.schema.names
# vectorassembler = VectorAssembler(inputCols = features, outputCol= 'assemblerfeatures')
# output_dataset = vectorassembler.transform(corr_sample)  

In [ ]:
# from pyspark.ml.stat import Correlation

# pearsonCorr = Correlation.corr(output_dataset, 'assemblerfeatures', 'pearson').collect()[0][0]

# #trasformo la DenseMatrix in un array numpy
# correlation_array = pearsonCorr.toArray() #ritorna un numpy.ndarray

# correlationDF = pd.DataFrame(
#     correlation_array,
#     index = features,
#     columns = features
# )

# sns.set(rc={'figure.figsize':(15,12)})
# sns.heatmap(correlationDF, annot=True)

# MachineLearning

Loss-Focused Prediction: Could you set up regression or classification models to predict if a batch tends to have higher brewing losses based on brew conditions (temperature, etc.)? This might provide insight on process adjustments to decrease loss percentage.
Efficiency Gains: What combinations of brewing factors have produced the highest Brewhouse Efficiency outcomes? Analyzing this could assist in streamlining processes for optimizing ingredient use.

## Intro

- Alcohol content

    - features = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", 
                 "Bitterness", "Color", "grains", "hops"]
    - Regression on alcohol content
    - Classficiation on alcohol content bin
    - Anomaly detection on alcohol content

- Loss during processes

- Quality score & usdperliter

In [67]:
sample = df.sample(fraction=0.1, seed=42) #.drop("Batch_ID")
print(f"Sample size: {sample.count()}")

Sample size: 1001591


In [68]:
sample.printSchema()

root
 |-- Batch_ID: long (nullable = true)
 |-- Brew_Date: string (nullable = true)
 |-- Beer_Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Fermentation_Time: long (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- pH_Level: double (nullable = true)
 |-- Gravity: double (nullable = true)
 |-- Alcohol_Content: double (nullable = true)
 |-- Bitterness: long (nullable = true)
 |-- Color: long (nullable = true)
 |-- Volume_Produced: long (nullable = true)
 |-- Total_Sales: double (nullable = true)
 |-- Quality_Score: double (nullable = true)
 |-- Brewhouse_Efficiency: double (nullable = true)
 |-- Loss_During_Brewing: double (nullable = true)
 |-- Loss_During_Fermentation: double (nullable = true)
 |-- Loss_During_Bottling_Kegging: double (nullable = true)
 |-- water: double (nullable = true)
 |-- grains: double (nullable = true)
 |-- hops: double (nullable = true)
 |-- USD_per_Liter: double (nullable = true)


In [69]:
sample.show(5)

+--------+-------------------+----------+-------+------------+-----------------+------------------+-----------------+------------------+-----------------+----------+-----+---------------+------------------+-----------------+--------------------+-------------------+------------------------+----------------------------+-----+------+----+-------------------+--------------------+-------------------------+------------------+-----------------------+
|Batch_ID|          Brew_Date|Beer_Style|    SKU|    Location|Fermentation_Time|       Temperature|         pH_Level|           Gravity|  Alcohol_Content|Bitterness|Color|Volume_Produced|       Total_Sales|    Quality_Score|Brewhouse_Efficiency|Loss_During_Brewing|Loss_During_Fermentation|Loss_During_Bottling_Kegging|water|grains|hops|      USD_per_Liter|   Brewing_efficency|encodedBeer_Stylefeatures|encodedSKUfeatures|encodedLocationfeatures|
+--------+-------------------+----------+-------+------------+-----------------+------------------+-----

## Alcohol content prediction task

### Regression

In [75]:
sample.select("Alcohol_content").toPandas().describe()

,Alcohol_content
count,1.001591e+06
mean,5.250250e+00
std,4.329737e-01
min,4.500002e+00
25%,4.875388e+00
50%,5.250748e+00
75%,5.625394e+00
max,5.999999e+00


#### !!!!

In [76]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


# Feature Preparation
features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity"]

# 1. Vector Assembler:
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
lr = LinearRegression(featuresCol="scaled_features", labelCol="Alcohol_Content") # FeaturesCol matches Scaler output
pipeline = Pipeline(stages=[assembler, scaler, lr])

Train Root Mean Squared Error (RMSE): 0.000000
Test Root Mean Squared Error (RMSE): -0.000000


AttributeError: 'CrossValidatorModel' object has no attribute 'stages'

In [ ]:
# Grid Search for Hyperparameter Tuning
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Cross Validation
evaluator = RegressionEvaluator(metricName="r2", labelCol="Alcohol_Content", predictionCol="prediction")

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

In [ ]:
# Dataset Preparation
regr_alc_sample = df.alias("regr_alc_sample")
regr_alc_train, regr_alc_test = regr_alc_sample.randomSplit([0.8, 0.2], seed=42)

# Fit the Pipelines:
model = crossval.fit(regr_alc_train)

In [ ]:
train_predictions = model.transform(regr_alc_train)
test_predictions = model.transform(regr_alc_test)
train_rmse = evaluator.evaluate(train_predictions)
test_rmse = evaluator.evaluate(test_predictions)

In [89]:
train_predictions.select("features", "scaled_features", "prediction").show(10, truncate=False)

+--------------------------------------------------------------+---------------------------------------------------------------------------------+----------------+
|features                                                      |scaled_features                                                                  |prediction      |
+--------------------------------------------------------------+---------------------------------------------------------------------------------+----------------+
|[13.0,17.21262685183783,4.641263904925896,1.0348629963531588] |[-0.5222037802457102,-0.9654347166770954,-1.2427668072627522,-1.3950099136753769]|5.24968525800582|
|[15.0,20.093420653681868,4.586376980370641,1.0630019931111474]|[0.17413831744063318,0.03226466097234418,-1.4329251606699431,0.5543303420883583] |5.24968525800582|
|[10.0,16.07617705149582,5.068217593669845,1.042327846990558]  |[-1.5667169267752252,-1.3590190328490022,0.23643449779382592,-0.8778793998941951]|5.24968525800582|
|[13.0,16.403160

In [90]:
train_predictions.select("features", "scaled_features", "prediction").show(10, truncate=False)

+--------------------------------------------------------------+---------------------------------------------------------------------------------+----------------+
|features                                                      |scaled_features                                                                  |prediction      |
+--------------------------------------------------------------+---------------------------------------------------------------------------------+----------------+
|[13.0,17.21262685183783,4.641263904925896,1.0348629963531588] |[-0.5222037802457102,-0.9654347166770954,-1.2427668072627522,-1.3950099136753769]|5.24968525800582|
|[15.0,20.093420653681868,4.586376980370641,1.0630019931111474]|[0.17413831744063318,0.03226466097234418,-1.4329251606699431,0.5543303420883583] |5.24968525800582|
|[10.0,16.07617705149582,5.068217593669845,1.042327846990558]  |[-1.5667169267752252,-1.3590190328490022,0.23643449779382592,-0.8778793998941951]|5.24968525800582|
|[13.0,16.403160

In [108]:
train_rmse, test_rmse

(5.440092820663267e-15, -7.511246269409355e-08)

In [111]:
# Evaluation: After getting predictions
print(f"Train R2: {train_rmse}")
print(f"Test R2: {test_rmse}")

# Access Coefficients
best_lr_model = model.bestModel.stages[-1]
best_lr_model.summary.meanSquaredError

Train R2: 5.440092820663267e-15
Test R2: -7.511246269409355e-08


0.18740155274656883

Interpreting Results

    RMSE: Evaluate the RMSE score. A lower value implies better average accuracy in predicting alcohol content using your features.
    
    Coefficients: Examine the sign (+/-) and magnitude of each feature's coefficient. These tell you:
        Positive Coefficient: Increases in this feature tend to associate with higher 'Alcohol_Content'.
        Negative Coefficient: Increases in this feature tend to associate with lower 'Alcohol_Content'.
        Magnitude: Larger the value (ignoring the sign), stronger the apparent impact of said feature, linearly speaking.


Interpreting the Results

    RMSE (0.432971): The context is crucial here. To judge whether this RMSE is "good" or "bad," consider the natural range of 'Alcohol_Content' values in your dataset.
        As a crude example, suppose 'Alcohol_Content' typically varies between 4% and 10%. Then a 0.43 error might be tolerable.
        If these alcohol values instead range from 4% to 5%, this same error becomes highly significant.

Analyzing Coefficients

Here's a breakdown of how to draw inferences from the model's coefficients:

    Sign:
        Positive coefficients suggest those features tend to be associated with higher alcohol content in your dataset.
        Negative coefficients imply an association with lower alcohol content.

    Magnitude: The larger the absolute value of a coefficient, the stronger its apparent linear influence on 'Alcohol_Content' is, all else being equal.

    Sparseness: Notice several coefficients (especially those for ingredient amounts) are very close to zero. This might mean these have practically negligible linear impact within the range of values encountered in your dataset.

Considerations

    Statistical Significance: This simple linear regression doesn't provide measures of statistical significance for each coefficient. Techniques exist to estimate confidence intervals, making more robust assertions about feature effect strength.

### Trees

Calculating Metrics
Python

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

y_true = regr_brewloss_test['Loss_During_Brewing'] # Actual Values
y_pred = test_predictions.select('prediction').collect() # Model Predictions

r2 = r2_score(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False) 
mae = mean_absolute_error(y_true, y_pred)

print('R-squared:', r2)
print('RMSE:', rmse)
print('MAE:', mae) 


Visualizing the Tree
Python

from sklearn import tree 
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8)) 
tree.plot_tree(your_dt_model, feature_names=features_cols, filled=True) 
plt.show()


Feature Importance
Python

feature_importances = your_dt_model.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1] # To show in descending order

print('Feature Importances:')
for f in range(len(features_cols)):
   print(features_cols[sorted_indices[f]], ':', feature_importances[sorted_indices[f]])


udf explanation site: https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/

#### Decision tree

In [ ]:
# from pyspark.ml.classification import DecisionTreeClassifier
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", 
#                  "Bitterness", "grains", "hops",]

# # Dataset preparation
# tree_alc_sample = sample.alias("tree_alc_sample")
# tree_alc_train, tree_alc_test = tree_alc_sample.randomSplit([0.8, 0.2], seed=42)

# # Binarizing 'Alcohol_Content'
# from pyspark.sql.functions import udf

# def binarize(alcohol_value):
#     if alcohol_value  <= 5.0:
#         return "Low"
#     elif alcohol_value <= 5.5:
#         return "Medium"
#     else:
#         return "High"

# binarize_udf = udf(binarize, StringType())
# tree_alc_train = tree_alc_train.withColumn("Alcohol_Bin", binarize_udf("Alcohol_Content"))
# tree_alc_test = tree_alc_test.withColumn("Alcohol_Bin", binarize_udf("Alcohol_Content"))

# # Encoding 'Alcohol_Bin' this solves the stringtype issue for the onehotencoder
# tree_alc_train = tree_alc_train.withColumn("Alcohol_Bin", 
#                         when(col("Alcohol_Bin") == "Low", 0). 
#                         when(col("Alcohol_Bin") == "Medium", 1).
#                         otherwise(2))

# tree_alc_test = tree_alc_test.withColumn("Alcohol_Bin", 
#                         when(col("Alcohol_Bin") == "Low", 0). 
#                         when(col("Alcohol_Bin") == "Medium", 1).
#                         otherwise(2)) 

# # Feature Preparation
# assembler = VectorAssembler(inputCols=features_cols, outputCol="features")  
# train_df_transformed = assembler.transform(tree_alc_train)
# test_df_transformed = assembler.transform(tree_alc_test)

# # Decision Tree Classifier
# dt = DecisionTreeClassifier(featuresCol="features", labelCol="Alcohol_Bin")
# dt_model = dt.fit(train_df_transformed)

# # Predictions
# predictions = dt_model.transform(test_df_transformed)

# # Evaluation (For Classification)
# evaluator = MulticlassClassificationEvaluator(labelCol="Alcohol_Bin", predictionCol="prediction", metricName="accuracy")
# accuracy = evaluator.evaluate(predictions)
# print("Accuracy: %f" % accuracy) 

# # Visualization (may need external libraries depending on your setup)
# print(dt_model.toDebugString) 


#### Decision Tree - pipeline crossval scaler

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator


features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", 
                 "Bitterness", "grains", "hops",]
# Dataset preparation
tree_alc_sample = df.alias("tree_alc_sample")
tree_alc_train, tree_alc_test = tree_alc_sample.randomSplit([0.8, 0.2], seed=42)

# Binarizing 'Alcohol_Content'
from pyspark.sql.functions import udf

def binarize(alcohol_value):
    if alcohol_value  <= 5.0:
        return "Low"
    elif alcohol_value <= 5.5:
        return "Medium"
    else:
        return "High"

binarize_udf = udf(binarize, StringType())
tree_alc_train = tree_alc_train.withColumn("Alcohol_Bin", binarize_udf("Alcohol_Content"))
tree_alc_test = tree_alc_test.withColumn("Alcohol_Bin", binarize_udf("Alcohol_Content"))

# Encoding 'Alcohol_Bin' this solves the stringtype issue for the onehotencoder
tree_alc_train = tree_alc_train.withColumn("Alcohol_Bin", 
                        when(col("Alcohol_Bin") == "Low", 0). 
                        when(col("Alcohol_Bin") == "Medium", 1).
                        otherwise(2))

tree_alc_test = tree_alc_test.withColumn("Alcohol_Bin", 
                        when(col("Alcohol_Bin") == "Low", 0). 
                        when(col("Alcohol_Bin") == "Medium", 1).
                        otherwise(2)) 

# Feature Preparation
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")  

# Standardization (Optional)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# Decision Tree Classifier
dt = DecisionTreeClassifier(featuresCol="features", labelCol="Alcohol_Bin")

# Parameter grid for tuning
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [3, 5, 8]) \
    .addGrid(dt.maxBins, [16, 32]) \
    .addGrid(dt.impurity, ['gini', 'entropy']) \
    .build()

# Cross Validation
evaluator = MulticlassClassificationEvaluator(labelCol="Alcohol_Bin", predictionCol="prediction", metricName="accuracy")
crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Pipeline for Modeling Stages Only
pipeline = Pipeline(stages=[assembler, crossval])

# Fit the Pipelines:
pipeline_model = pipeline.fit(tree_alc_train)

# Evaluation: After getting predictions
train_predictions = pipeline_model.transform(tree_alc_train)
test_predictions = pipeline_model.transform(tree_alc_test)

train_accuracy = evaluator.evaluate(train_predictions)
test_accuracy = evaluator.evaluate(test_predictions)

print("Train Accuracy: %f" % train_accuracy)
print("Test Accuracy: %f" % test_accuracy)

# Access Best Model
best_dt_model = pipeline_model.stages[1].bestModel
print(best_dt_model.toDebugString)

#### Random Forest - pipeline crossval scaler

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator


features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", 
                 "Bitterness", "grains", "hops",]

# Dataset preparation
tree_alc_sample = df.alias("tree_alc_sample")
tree_alc_train, tree_alc_test = tree_alc_sample.randomSplit([0.8, 0.2], seed=42)

# Binarizing 'Alcohol_Content'
from pyspark.sql.functions import udf

def binarize(alcohol_value):
    if alcohol_value  <= 5.0:
        return "Low"
    elif alcohol_value <= 5.5:
        return "Medium"
    else:
        return "High"

binarize_udf = udf(binarize, StringType())
tree_alc_train = tree_alc_train.withColumn("Alcohol_Bin", binarize_udf("Alcohol_Content"))
tree_alc_test = tree_alc_test.withColumn("Alcohol_Bin", binarize_udf("Alcohol_Content"))

# Encoding 'Alcohol_Bin' this solves the stringtype issue for the onehotencoder
tree_alc_train = tree_alc_train.withColumn("Alcohol_Bin", 
                        when(col("Alcohol_Bin") == "Low", 0). 
                        when(col("Alcohol_Bin") == "Medium", 1).
                        otherwise(2))

tree_alc_test = tree_alc_test.withColumn("Alcohol_Bin", 
                        when(col("Alcohol_Bin") == "Low", 0). 
                        when(col("Alcohol_Bin") == "Medium", 1).
                        otherwise(2)) 

# Feature Preparation
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")  

# Standardization (Optional)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# Decision Tree Classifier
rf = RandomForestClassifier(labelCol="Alcohol_Bin")

# Parameter grid for tuning
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [3, 5, 8]) \
    .addGrid(rf.maxBins, [16, 32]) \
    .addGrid(rf.impurity, ['gini', 'entropy']) \
    .addGrid(rf.numTrees, [20, 50]) \
    .addGrid(rf.subsamplingRate, [0.6, 0.8]) \
    .addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt']) \
    .build()

# Cross Validation
evaluator = MulticlassClassificationEvaluator(labelCol="Alcohol_Bin", predictionCol="prediction", metricName="accuracy")
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Pipeline for Modeling Stages Only
pipeline = Pipeline(stages=[assembler, scaler, crossval])

# Fit the Pipelines:
pipeline_model = pipeline.fit(tree_alc_train)

# Evaluation: After getting predictions
train_predictions = pipeline_model.transform(tree_alc_train)
test_predictions = pipeline_model.transform(tree_alc_test)

train_accuracy = evaluator.evaluate(train_predictions)
test_accuracy = evaluator.evaluate(test_predictions)

print("Train Accuracy: %f" % train_accuracy)
print("Test Accuracy: %f" % test_accuracy)

# Access Best Model
best_dt_model = pipeline_model.stages[2].bestModel
print(best_dt_model.toDebugString)

### Anomaly detection - NOT DONE

Standardization: To Scale or Not to Scale?

    For Isolation Forests: Tree-based algorithms are generally less sensitive to feature scaling than distance-based ones like K-means. There might be a marginal benefit but it's not strictly mandatory.
    Potential Benefit: Some of your features have relatively tight standard deviations ('Alcohol_Content', 'Gravity'), while others like 'Bitterness' and 'Volume_Produced' have more extensive spreads. Standardization could prevent features with larger scales from overpowering the model's analysis, especially if correlations exist.
    Recommendation: I suggest experimenting with both standardized and non-standardized data with your Isolation Forest model. Compare the identified anomalies; if there's significant discrepancy, standardization likely improved interpretation.


In [ ]:
# from pyspark.ml.feature import VectorAssembler, StandardScaler

# scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False) 
# scalerModel = scaler.fit(df_transformed)
# scaled_data = scalerModel.transform(df_transformed)

In [ ]:
# from pyspark.ml.classification import DecisionTreeClassifier, IsolationForest
# from pyspark.ml.feature import VectorAssembler, StandardScaler
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", 
#                  "Bitterness", "Color", "grains", "hops", "encodedBeer_Stylefeatures"]

# tree_alc_sample = sample.alias("tree_alc_sample")

# # Binarizing 'Alcohol_Content'
# from pyspark.sql.functions import udf

# def binarize(alcohol_value):
#     if alcohol_value  <= 5.0:
#         return "Low"
#     elif alcohol_value <= 5.5:
#         return "Medium"
#     else:
#         return "High"

# binarize_udf = udf(binarize, StringType())
# tree_alc_sample = tree_alc_sample.withColumn("Alcohol_Bin", binarize_udf("Alcohol_Content"))

# # Encoding 'Alcohol_Bin' this solves the stringtype issue for the onehotencoder
# tree_alc_sample = tree_alc_sample.withColumn("Alcohol_Bin", 
#                         when(col("Alcohol_Bin") == "Low", 0). 
#                         when(col("Alcohol_Bin") == "Medium", 1).
#                         otherwise(2)) 

# # Feature Preparation
# assembler = VectorAssembler(inputCols=features_cols, outputCol="features")  
# df_transformed = assembler.transform(tree_alc_sample)

# # Decision Tree Classifier
# iforest = IsolationForest(featuresCol="features", labelCol="Alcohol_Bin")



## Loss prediction task

### Loss during brewing

#### first features batch = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", "Volume_Produced", "Brewhouse_Efficiency"]

lr (r2) = 
- Train Root Mean Squared Error (r2): 0.000000 
- Test Root Mean Squared Error (r2): -0.000015
- [0.0,0.0,0.0,0.0,0.0,0.0]

lr (mae) =
- Train Root Mean Squared Error (mae): 0.999923
- Test Root Mean Squared Error (mae): 0.998363
- [0.0,0.0,0.0,0.0,0.0,0.0]

dt (r2) = 
- Train Root Mean Squared Error (r2): 0.000203
- Test Root Mean Squared Error (r2): -0.000283

dt (mae) =
- Train Root Mean Squared Error (mae): 0.999888
- Test Root Mean Squared Error (mae): 0.998390


#### second feature bacth = ["Volume_Produced","Total_Sales", "Quality_Score", "Brewhouse_Efficiency"]

lr (r2) =
- Train Root Mean Squared Error (r2): 0.000005
- Test Root Mean Squared Error (r2): -0.000021
- [3.8205088347048814e-07,2.258462545652026e-07,-0.0005469537426189472,-9.910365816318696e-05]

lr (mae) = 
- Train Root Mean Squared Error (mae): 0.999918
- Test Root Mean Squared Error (mae): 0.998364
- [6.560648378543714e-07,3.879291685535881e-07,-0.0009406175959918453,-0.00017003566917370277]

dt (r2) = 
- Train Root Mean Squared Error (r2): 0.000040
- Test Root Mean Squared Error (r2): -0.000072

dt (mae) =
- Train Root Mean Squared Error (mae): 0.999895
- Test Root Mean Squared Error (mae): 0.998379

gbt (r2) = 
- Train Root Mean Squared Error (r2): 0.000126
- Test Root Mean Squared Error (r2): -0.000090

#### code

In [ ]:
df.show(5)

In [ ]:
from pyspark.ml.regression import LinearRegression, GBTRegressor, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler, RobustScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Feature Preparation
features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", "Brewhouse_Efficiency"]

# Dataset Preparation
regr_brewloss_sample = df.alias("regr_brewloss_sample")

# Train-Test Split
regr_brewloss_train, regr_brewloss_test = regr_brewloss_sample.randomSplit([0.8, 0.2], seed=42)

# 1. Vector Assembler:
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")

# 2. Standard Scaler (If you decide to standardize):
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
# scaler = RobustScaler(inputCol="features", outputCol="scaled_features")

# 3. Regression Models:
lr = LinearRegression(featuresCol="features", labelCol="Loss_During_Brewing")
# dt = DecisionTreeRegressor(featuresCol="features", labelCol="Loss_During_Brewing")
# rf = RandomForestRegressor(featuresCol="features", labelCol="Loss_During_Brewing")
# gbt = GBTRegressor(featuresCol="features", labelCol="Loss_During_Brewing")

# Grid Search for Hyperparameter Tuning

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Cross Validation
evaluator = RegressionEvaluator(metricName="mae", labelCol="Loss_During_Brewing", predictionCol="prediction")
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Pipeline for Modeling Stages Only
pipeline = Pipeline(stages=[assembler,scaler, crossval])

# Fit the Pipelines:
pipeline_model = pipeline.fit(regr_brewloss_train)

# Evaluation: After getting predictions
train_predictions = pipeline_model.transform(regr_brewloss_train)
test_predictions = pipeline_model.transform(regr_brewloss_test)
train_rmse = evaluator.evaluate(train_predictions)
test_rmse = evaluator.evaluate(test_predictions)

print("Train Root Mean Squared Error (mae): %f" % train_rmse)
print("Test Root Mean Squared Error (mae): %f" % test_rmse)

# Access Best Model
best_lr_model = pipeline_model.stages[2].bestModel
print(best_lr_model.coefficients)

print(pipeline_model.stages)

### Loss during fermentation

#### first features batch = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", "Volume_Produced", "Brewhouse_Efficiency"]

lr (r2) = 
- Train Root Mean Squared Error (r2): 0.000000
- Test Root Mean Squared Error (r2): -0.000002

lr (mae) = 
- Train Root Mean Squared Error (mae): 0.999648
- Test Root Mean Squared Error (mae): 1.002373


dt (r2) = 
- Train Root Mean Squared Error (r2): -0.000026
- Test Root Mean Squared Error (r2): -0.000016

dt (mae) =
- Train Root Mean Squared Error (mae): 0.999651
- Test Root Mean Squared Error (mae): 1.002360

#### second feature bacth = ["Volume_Produced","Total_Sales", "Quality_Score", "Brewhouse_Efficiency"]

#### code

In [ ]:
df.show(5)

In [ ]:
from pyspark.ml.regression import LinearRegression, GBTRegressor, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Feature Preparation
features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", "Volume_Produced", "Brewhouse_Efficiency"]

# Dataset Preparation
regr_fermloss_sample = df.alias("regr_fermloss_sample")

# Train-Test Split
regr_fermloss_train, regr_fermloss_test = regr_fermloss_sample.randomSplit([0.8, 0.2], seed=42)

# 1. Vector Assembler:
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")

# 2. Standard Scaler (If you decide to standardize):
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# 3. Regression Models:
lr = LinearRegression(featuresCol="features", labelCol="Loss_During_Fermentation")
# dt = DecisionTreeRegressor(featuresCol="features", labelCol="Loss_During_Brewing")
# rf = RandomForestRegressor(featuresCol="features", labelCol="Loss_During_Brewing")
# gbt = GBTRegressor(featuresCol="features", labelCol="Loss_During_Brewing")

# Grid Search for Hyperparameter Tuning

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Cross Validation
evaluator = RegressionEvaluator(metricName="mae", labelCol="Loss_During_Fermentation", predictionCol="prediction")
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Pipeline for Modeling Stages Only
pipeline = Pipeline(stages=[assembler,scaler, crossval])

# Fit the Pipelines:
pipeline_model = pipeline.fit(regr_brewloss_train)

# Evaluation: After getting predictions
train_predictions = pipeline_model.transform(regr_fermloss_train)
test_predictions = pipeline_model.transform(regr_fermloss_test)
train_rmse = evaluator.evaluate(train_predictions)
test_rmse = evaluator.evaluate(test_predictions)

print("Train Root Mean Squared Error (mae): %f" % train_rmse)
print("Test Root Mean Squared Error (mae): %f" % test_rmse)

# # Access Best Model
# best_lr_model = pipeline_model.stages[2].bestModel
# print(best_lr_model.coefficients)

## beer type assessment task

In [ ]:
from pyspark.ml.regression import LinearRegression, GBTRegressor, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Feature Preparation
features_cols = ["Fermentation_Time" ,"Temperature", "pH_Level" , "Gravity", "Volume_Produced", "Brewhouse_Efficiency"]

# Dataset Preparation
regr_fermloss_sample = df.alias("regr_fermloss_sample")

# Train-Test Split
regr_fermloss_train, regr_fermloss_test = regr_fermloss_sample.randomSplit([0.8, 0.2], seed=42)

# 1. Vector Assembler:
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")

# 2. Standard Scaler (If you decide to standardize):
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# 3. Regression Models:
lr = LinearRegression(featuresCol="features", labelCol="Loss_During_Fermentation")
# dt = DecisionTreeRegressor(featuresCol="features", labelCol="Loss_During_Brewing")
# rf = RandomForestRegressor(featuresCol="features", labelCol="Loss_During_Brewing")
# gbt = GBTRegressor(featuresCol="features", labelCol="Loss_During_Brewing")

# Grid Search for Hyperparameter Tuning

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Cross Validation
evaluator = RegressionEvaluator(metricName="mae", labelCol="Loss_During_Fermentation", predictionCol="prediction")
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Pipeline for Modeling Stages Only
pipeline = Pipeline(stages=[assembler,scaler, crossval])

# Fit the Pipelines:
pipeline_model = pipeline.fit(regr_brewloss_train)

# Evaluation: After getting predictions
train_predictions = pipeline_model.transform(regr_fermloss_train)
test_predictions = pipeline_model.transform(regr_fermloss_test)
train_rmse = evaluator.evaluate(train_predictions)
test_rmse = evaluator.evaluate(test_predictions)

print("Train Root Mean Squared Error (mae): %f" % train_rmse)
print("Test Root Mean Squared Error (mae): %f" % test_rmse)

# # Access Best Model
# best_lr_model = pipeline_model.stages[2].bestModel
# print(best_lr_model.coefficients)

# Time series

## prep

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [ ]:
series_df = df.repartition('brew_date') \
              .orderBy('brew_date')

In [ ]:
target_column = 'total_sales'
from pyspark.sql.types import DateType

brew_data = series_df.groupBy('brew_date').agg(F.mean(target_column).alias('avg_sales'))
brew_data = brew_data.withColumn('brew_date', F.col('brew_date').cast(DateType()))  

In [ ]:
brew_data.printSchema()

In [ ]:
brew_data.show(5)

In [ ]:
from pyspark.sql.window import Window

window_size = 7  # 7 day moving average
windowSpec = Window.orderBy('brew_date').rowsBetween(-window_size + 1, 0)  # Look back window
result_df = brew_data.withColumn('forecast', F.avg('avg_sales').over(windowSpec))
result_df.show()